In [1]:
import pandas as pd
import numpy as np
import pickle
import gensim
from collections import Counter, defaultdict

In [2]:
import sys
import os
def add_sys_path(p):
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

add_sys_path('../')

In [3]:
from evaluation import evaluate

In [4]:
import data_split

In [5]:
from evaluation.evaluate import read_dataset

In [6]:
import json

In [7]:
from tqdm.auto import tqdm, trange

In [8]:
from sklearn.neighbors import KDTree

# Load w2v models

In [9]:
import my_knn
from importlib import reload
reload(my_knn)

<module 'my_knn' from 'D:\\dialogue2020\\diachronic-wordnets\\code\\dale\\my_knn.py'>

In [10]:
w2v = gensim.models.KeyedVectors.load_word2vec_format(  #load_word2vec_format
    '../baselines/models/model.bin',
    binary=True,
    unicode_errors='ignore',
)

In [14]:
add_pos = True
n = 300

In [15]:
w2v_embedder = my_knn.W2VWrapper(w2v, n=n, add_pos=add_pos)

In [16]:
w2v_embedder_pos = my_knn.W2VWrapper(w2v,  n=n, add_pos=add_pos, pos_weights={'NOUN': 1.0, 'PREP': 0.1}, default_weight=0.5)

In [17]:
w2v_embedder_pos_vb = my_knn.W2VWrapper(w2v,  n=n, add_pos=add_pos, pos_weights={'VERB': 1.0, 'PREP': 0.1}, default_weight=0.5)

# Evaluation

## Nouns

In [25]:
def w2v_scorer(text1, text2):
    return np.dot(w2v_embedder.get_text_vec(text1), w2v_embedder.get_text_vec(text2)) ** 5

In [19]:
public_test_verbs = pd.read_csv('../../datasets/ru/nouns_private_no_labels.tsv', header=None)
public_test_verbs.columns = ['text']

In [21]:
full_syn_storage, full_rel_storage, full_rel_df = my_knn.prepare_storages(
    synsets_filename='../../datasets/ruwordnet/synsets.N.xml',
    relations_filename='../../datasets/ruwordnet/synset_relations.N.xml',
    forbidden_words=set()
)

number of texts: 86549
forbidden senses are 0
numer of ids 29296 long list is 119273



28118
555
5
37249


In [22]:
full_w2v_vecs = np.stack([w2v_embedder(t) for t in tqdm(full_syn_storage.texts_long) ])
full_w2v_tree = KDTree(full_w2v_vecs)

In [23]:
full_w2v_vecs_pos = np.stack([w2v_embedder_pos(t) for t in tqdm(full_syn_storage.texts_long) ])
full_w2v_tree_pos = KDTree(full_w2v_vecs_pos)

In [26]:
public_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=full_w2v_tree_pos, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage, rel_storage=full_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    )  
    for txt in tqdm(public_test_verbs.text)
}

In [27]:
sub = my_knn.dict2submission(public_test_hypos, full_syn_storage.id2synset)
sub

,noun,result,result_text
0,АБСЕНТЕИЗМ,548-N,АЛКОГОЛЬНЫЙ НАПИТОК
1,АБСЕНТЕИЗМ,108336-N,КРЕПКИЙ АЛКОГОЛЬНЫЙ НАПИТОК
2,АБСЕНТЕИЗМ,549-N,НАПИТОК
3,АБСЕНТЕИЗМ,8367-N,ВИНО
4,АБСЕНТЕИЗМ,142362-N,СПИРТОСОДЕРЖАЩАЯ ПРОДУКЦИЯ
5,АБСЕНТЕИЗМ,3675-N,ЖИДКОСТЬ
6,АБСЕНТЕИЗМ,9419-N,ОДУРМАНИВАЮЩЕЕ ВЕЩЕСТВО
7,АБСЕНТЕИЗМ,550-N,БЕЗАЛКОГОЛЬНЫЕ НАПИТКИ
8,АБСЕНТЕИЗМ,368-N,ПРОДУКТЫ ПИТАНИЯ
9,АБСЕНТЕИЗМ,108232-N,ДЕСЕРТНОЕ ВИНО


In [35]:
sub.to_csv('../baselines/predictions/nouns_private_dale.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АДСОРБИРОВАТЬ,106950-V,"ПРЕВРАТИТЬ (ПРИДАТЬ ИНОЙ ВИД, КАЧЕСТВО)"
1,АДСОРБИРОВАТЬ,114441-V,ОЧИСТИТЬ ОТ ПРИСУТСТВИЯ
2,АДСОРБИРОВАТЬ,111097-V,ИЗМЕНЕНИЕ АГРЕГАТНОГО СОСТОЯНИЯ
3,АДСОРБИРОВАТЬ,106629-V,"ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ"
4,АДСОРБИРОВАТЬ,120784-V,"ПОЛУЧИТЬ (ДОБЫТЬ, ПРОИЗВЕСТИ)"
5,АДСОРБИРОВАТЬ,128431-V,ОБЕЗВРЕДИТЬ
6,АДСОРБИРОВАТЬ,106483-V,"УНИЧТОЖИТЬ, ПРЕКРАТИТЬ СУЩЕСТВОВАНИЕ"
7,АДСОРБИРОВАТЬ,106631-V,"ИЗМЕНИТЬ, СДЕЛАТЬ ИНЫМ"
8,АДСОРБИРОВАТЬ,106477-V,"ПОЯВИТЬСЯ, ВОЗНИКНУТЬ, НАЧАТЬСЯ"
9,АДСОРБИРОВАТЬ,106785-V,"ПРЕОБРАЗОВАТЬ, ПЕРЕДЕЛАТЬ"


## Verbs

In [29]:
public_test_verbs = pd.read_csv('../../datasets/ru/verbs_private_no_labels.tsv', header=None)
public_test_verbs.columns = ['text']

In [31]:
full_syn_storage_v, full_rel_storage_v, full_rel_df_v = my_knn.prepare_storages(
    synsets_filename='../../datasets/ruwordnet/synsets.V.xml',
    relations_filename='../../datasets/ruwordnet/synset_relations.V.xml',
    forbidden_words=set()
)

number of texts: 32822
forbidden senses are 0
numer of ids 7521 long list is 49002


7408
218
5
10317


In [32]:
full_w2v_vecs_pos_v = np.stack([w2v_embedder_pos_vb(t) for t in tqdm(full_syn_storage_v.texts_long) ])
full_w2v_tree_pos_v = KDTree(full_w2v_vecs_pos_v)

In [33]:
public_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree_pos_v, text2vec=w2v_embedder_pos_vb,
        synset_storage=full_syn_storage_v, rel_storage=full_rel_storage_v,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    )  
    for txt in tqdm(public_test_verbs.text)
}

In [36]:
sub = my_knn.dict2submission(public_test_hypos, full_syn_storage_v.id2synset)
sub

,noun,result,result_text
0,АДСОРБИРОВАТЬ,106950-V,"ПРЕВРАТИТЬ (ПРИДАТЬ ИНОЙ ВИД, КАЧЕСТВО)"
1,АДСОРБИРОВАТЬ,114441-V,ОЧИСТИТЬ ОТ ПРИСУТСТВИЯ
2,АДСОРБИРОВАТЬ,111097-V,ИЗМЕНЕНИЕ АГРЕГАТНОГО СОСТОЯНИЯ
3,АДСОРБИРОВАТЬ,106629-V,"ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ"
4,АДСОРБИРОВАТЬ,120784-V,"ПОЛУЧИТЬ (ДОБЫТЬ, ПРОИЗВЕСТИ)"
5,АДСОРБИРОВАТЬ,128431-V,ОБЕЗВРЕДИТЬ
6,АДСОРБИРОВАТЬ,106483-V,"УНИЧТОЖИТЬ, ПРЕКРАТИТЬ СУЩЕСТВОВАНИЕ"
7,АДСОРБИРОВАТЬ,106631-V,"ИЗМЕНИТЬ, СДЕЛАТЬ ИНЫМ"
8,АДСОРБИРОВАТЬ,106477-V,"ПОЯВИТЬСЯ, ВОЗНИКНУТЬ, НАЧАТЬСЯ"
9,АДСОРБИРОВАТЬ,106785-V,"ПРЕОБРАЗОВАТЬ, ПЕРЕДЕЛАТЬ"


In [38]:
sub.to_csv('../baselines/predictions/verbs_private_dale.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АДСОРБИРОВАТЬ,106950-V,"ПРЕВРАТИТЬ (ПРИДАТЬ ИНОЙ ВИД, КАЧЕСТВО)"
1,АДСОРБИРОВАТЬ,114441-V,ОЧИСТИТЬ ОТ ПРИСУТСТВИЯ
2,АДСОРБИРОВАТЬ,111097-V,ИЗМЕНЕНИЕ АГРЕГАТНОГО СОСТОЯНИЯ
3,АДСОРБИРОВАТЬ,106629-V,"ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ"
4,АДСОРБИРОВАТЬ,120784-V,"ПОЛУЧИТЬ (ДОБЫТЬ, ПРОИЗВЕСТИ)"
5,АДСОРБИРОВАТЬ,128431-V,ОБЕЗВРЕДИТЬ
6,АДСОРБИРОВАТЬ,106483-V,"УНИЧТОЖИТЬ, ПРЕКРАТИТЬ СУЩЕСТВОВАНИЕ"
7,АДСОРБИРОВАТЬ,106631-V,"ИЗМЕНИТЬ, СДЕЛАТЬ ИНЫМ"
8,АДСОРБИРОВАТЬ,106477-V,"ПОЯВИТЬСЯ, ВОЗНИКНУТЬ, НАЧАТЬСЯ"
9,АДСОРБИРОВАТЬ,106785-V,"ПРЕОБРАЗОВАТЬ, ПЕРЕДЕЛАТЬ"
